In [29]:
import secrets

import ed25519
import x25519
import hkdf
import gcm

In [30]:
A_seed_k = secrets.token_bytes(32)

A_pk = ed25519.pk(A_seed_k)
# And makes A_pk public

In [31]:
B_seed_k = secrets.token_bytes(32)

B_pk = ed25519.pk(B_seed_k)
# And makes B_pk public

In [32]:
a = secrets.token_bytes(32)

A = x25519.mult(a, 9)
sig_A = ed25519.sign(A_seed_k, A)
# Sends (A, sig_A) to B     # TODO: How to actually send them together

In [33]:
b = secrets.token_bytes(32)

B = x25519.mult(b, 9)
sig_B = ed25519.sign(B_seed_k, B)
# Send (B, sig_B) to A

In [34]:
# Verify message came from B
def get_common_key(a: int, B: bytes, sig_B: bytes):
    if ed25519.verify(B_pk, B, sig_B) == True:
        common_key = x25519.mult(a, B)
    return common_key

In [35]:
if ed25519.verify(B_pk, B, sig_B) == True:
    A_shared_secret = x25519.mult(a, B)
else:
    raise ValueError("Signiture doesn't verify")

print("Shared secret:", A_shared_secret)

A_session_key = hkdf.hkdf(A_shared_secret, bytes([0x00]), "session key".encode(), L=32)

print("Derived session key:", A_session_key)

Shared secret: b"\x0e'\xc4\xd6\xf2\x7ff\x04\x0f\xe2{\x0c%M\xcf\x9d\xcf\x15e\x954\xca\x9c\x0c\xcb^6\x19\x18eJ\n"
Derived session key: b'\xd7\xf0"\xe4\x03\xde\x96\xb9\x0f\xecylc\xa6\xc9L\x00\xd6Q\x00\xb5\xacW\xbf\x95\xb1\xcd\xb2nG8\x10'


In [36]:
if ed25519.verify(A_pk, A, sig_A) == True:
    B_shared_secret = x25519.mult(b, A)
else:
    raise ValueError("Signiture doesn't verify")

print("Shared secret:", B_shared_secret)

B_session_key = hkdf.hkdf(B_shared_secret, bytes([0x00]), "session key".encode(), L=32)

print("Derived session key:", B_session_key)

Shared secret: b"\x0e'\xc4\xd6\xf2\x7ff\x04\x0f\xe2{\x0c%M\xcf\x9d\xcf\x15e\x954\xca\x9c\x0c\xcb^6\x19\x18eJ\n"
Derived session key: b'\xd7\xf0"\xe4\x03\xde\x96\xb9\x0f\xecylc\xa6\xc9L\x00\xd6Q\x00\xb5\xacW\xbf\x95\xb1\xcd\xb2nG8\x10'


In [37]:
print("Are the derived keys the same?: ", A_session_key == B_session_key)

Are the derived keys the same?:  True


In [44]:
ciphertext, tag = gcm.encrypt("Ana are mere".encode(), A_session_key, secrets.token_bytes(16), None)
print(ciphertext, tag)
# Send (ciphertext, tag) to B

b'\ng\xec\xe6\x95\xc1~P\xbd\xfe\x86\xbb}\x84P\xa7\x1eGf\x86\xb5\xb8\t\xdd\xf0\xa0\xd3\x13@7\xbc\x85' b'%\x85\xa1\x93\xc5\x8e\xd0\xbd~B\x18\xc3\x05\xb4\x91\x9a'


In [39]:
plaintext = gcm.decrypt(ciphertext, B_session_key, tag, None).decode()
print(plaintext)

Ana are mere
